In [108]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog


# Open a file selection dialog and get the selected file's path
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

# Load the dataframe
df = pd.read_excel(file_path, sheet_name="GISLeaseUpdateResults")
df.head(5)

,Internal ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date (Letter Merge),End Date (Letter Merge),Related Asset,Name,Acreage,Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section#,Lease Status
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,70.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,640.00,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,250.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,637.12,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,609.18,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active


In [109]:
# Change Column Names
column_name_changes = {'Internal ID': 'Netsuite Lease ID',
                      'Start Date (Letter Merge)': 'Start Date',
                      'End Date (Letter Merge)': 'End Date',
                      'Related Asset': 'Related FAM(s)',
                      'Name': 'Related FAM Name',
                      'Legal Description': 'Lease Legal Description',
                      'Section#': 'Section'}

# Use the rename method to change the column names
df = df.rename(columns=column_name_changes)

df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,Acreage,Lease Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,70.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,640.00,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,250.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,637.12,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,609.18,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active


In [110]:
# Group the dataframe by the Lease Number column and sum the values in the Acreage column
summary_df = df.groupby('Transaction Number')['Acreage'].sum().reset_index()

# Rename the Acreage column to Total Lease Acreage
summary_df = summary_df.rename(columns={'Acreage': 'Total Lease Acreage'})

# The summary dataframe now contains the total acreage for each Lease Number
summary_df

,Transaction Number,Total Lease Acreage
0,AG-100001,960.000
1,AG-100014,47859.790
2,AG-100112,2660.550
3,AG-100145,1280.000
4,AG-100148,5175.020
...,...,...
4176,TOW-112838,40.000
4177,TOW-113440,0.028
4178,TOW-113960,160.000
4179,TOW-114153,0.370


In [111]:
df = pd.merge(df, summary_df, on='Transaction Number')

df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status,Total Lease Acreage
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active,960.00
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active,47859.79
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active,47859.79


In [112]:
# Groupby Lease Legal Descriptions by Transaction number. 

df_legals = df.groupby('Transaction Number')['Lease Legal Description'].apply(lambda x: ', '.join([str(i) for i in x])).reset_index()
df_legals.columns = ['Transaction Number', 'Lease Legal Description']

# Nan values need to be dropped
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace('nan, ', '')
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace('nan', '')
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace(', nan', '')

# Replace values with "See Lease Documentation" if greater than 50
mask = df_legals['Lease Legal Description'].str.len() > 50
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].where(~mask, 'See Lease Documentation')

df_legals


,Transaction Number,Lease Legal Description
0,AG-100001,"E2, ALL, E2"
1,AG-100014,See Lease Documentation
2,AG-100112,See Lease Documentation
3,AG-100145,"ALL, ALL, ALL, ALL"
4,AG-100148,See Lease Documentation
...,...,...
4176,TOW-112838,ALL
4177,TOW-113440,SESWSE
4178,TOW-113960,NW
4179,TOW-114153,NENESE


In [113]:
# Merge df and df_legals into merged_df
merged_df = pd.merge(df, df_legals, on='Transaction Number', how='outer')
merged_df = merged_df.rename(columns={'Lease Legal Description_y': 'Lease Legal Description(s)'})

merged_df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status,Total Lease Acreage,Lease Legal Description(s)
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00,"E2, ALL, E2"
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,36,Active,960.00,"E2, ALL, E2"
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00,"E2, ALL, E2"
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,15S,62W,1,Active,47859.79,See Lease Documentation
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,16S,62W,1,Active,47859.79,See Lease Documentation


In [114]:
df.fillna("",inplace=True)
grouped_df = df.groupby('Transaction Number')['Section'].apply(lambda x: ', '.join(filter(None, [val.strip() for val in x])) if x.nunique()>1 else x.iloc[0]).reset_index()

# Replace values with "See Lease Documentation" if greater than 20
mask = grouped_df['Section'].str.len() > 20
grouped_df['Section'] = grouped_df['Section'].where(~mask, 'See Lease Documentation')

grouped_df.sample(50)


,Transaction Number,Section
2936,OG-112970,16
3170,OG-114629,20
3793,SM-102489,16
3028,OG-113600,6
1079,AG-110351,"19, 20, 19, 20"
2656,OG-108516,See Lease Documentation
381,AG-102064,36
717,AG-109384,"16, 28"
1598,AG-112794,"36, 36"
2524,OG-108331,16


In [115]:
# Rename Section to Section(s) in grouped_df and add to merged_df
grouped_df.rename(columns={'Section': 'Section(s)'}, inplace=True)
merged_df = pd.merge(merged_df, grouped_df[['Transaction Number', 'Section(s)']], on='Transaction Number', how='left')

In [116]:
# Check if Section is dropped
if 'Section' in merged_df.columns:
    merged_df.drop(columns=['Section'], inplace=True)


merged_df.head(10)


,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Lease Status,Total Lease Acreage,Lease Legal Description(s),Section(s)
34795,879828,Agriculture,Grazing,AG-113877,"GREGORY S EMICK, XS RANCH, LLC, VALERIE L EMICK",NaN,NaT,NaT,FAM006311,6-27S-47W-36-Surface-Prowers-School,...,10.0,Eric Van Dyk,Southeast,6,27S,47W,Active,640.00,ALL,36
28425,300715,Agriculture,Grazing,AG-110226,"VISINTAINER SHEEP COMPANY, A COLORADO LIMITED ...",NaN,2016-10-01,2026-10-01,FAM003107,6-9N-90W-32-Surface-Moffat-School,...,10.0,Jerod Smith,Northwest,6,9N,90W,Active,6969.52,See Lease Documentation,See Lease Documentation
21756,19512,Oil and Gas,Oil & Gas,OG-108553,VERDAD RESOURCES LLC,OG 8436 05,2005-02-17,NaT,FAM035895,6-9N-60W-16-OGWell-Weld-School-05-123-31642,...,5.0,Catie Stitt,North Central,6,9N,60W,Active,1280.00,"ALL, ALL, ALL, ALL, ALL, ALL, ALL, ALL",16
9780,19207,Oil and Gas,Oil & Gas,OG-108248,"OWN RESOURCES ROCKIES, LLC",OG 3227 95,1995-11-16,NaT,FAM038400,6-2S-44W-36-OGWell-Yuma-School-05-125-11128,...,5.0,Catie Stitt,Northeast,6,2S,44W,Active,320.00,"NE,SE, NE,SE, NE,SE, NE,SE, NE,SE, NE,SE, NE,SE,",See Lease Documentation
18116,19433,Oil and Gas,Oil & Gas,OG-108474,"NOBLE ENERGY, INC.",OG 7885 70,1968-11-20,NaT,FAM036735,6-3N-64W-22-OGWell-Weld-School-05-123-07660,...,5.0,Catie Stitt,North Central,6,3N,64W,Active,960.00,See Lease Documentation,22
33265,829189,Recreation,NaN,REC-113500,COLO DIV PARKS & WILDLIFE/PUBLIC ACCESS PROGRAM,NaN,NaT,2023-06-30,FAM004039,6-12S-73W-28-Surface-Park-Internal Improvements,...,NaN,Abraham Medina,NaN,6,12S,73W,Active,207878.20,See Lease Documentation,See Lease Documentation
35625,978593,Geologic Carbon Storage,NaN,GCS-114345,"CARBON AMERICA, INC : DENOVA SEQUESTRATION, LLC",NaN,NaT,2025-01-01,FAM007733,6-1N-49W-30-Other Minerals-Washington-School,...,3.0,Ben Teschner,Northeast,6,1N,49W,Active,10241.51,See Lease Documentation,See Lease Documentation
26249,37469,Oil and Gas,Oil & Gas,OG-109252,"FUNDARE REDTAIL, LLC",1004 09,2009-08-20,NaT,FAM036274,6-10N-57W-16-OGWell-Weld-School-05-123-38975,...,5.0,Catie Stitt,North Central,6,10N,57W,Active,1280.00,"ALL, ALL",See Lease Documentation
25738,21622,Oil and Gas,Oil & Gas,OG-109022,KERR-MCGEE OIL & GAS ONSHORE LP,NaN,2014-08-08,NaT,FAM039919,6-2N-68W-3-Weld-Internal Improvements-05-123-4...,...,5.0,Catie Stitt,North Central,6,2N,68W,Active,197.64,See Lease Documentation,3
3012,13044,Solid Mineral,Long Term Non Development Minerals,SM-102663,ROSETTA DUNCAN,LT 363,NaT,NaT,FAM031931,6-4S-64W-16-Other Minerals-Arapahoe-School,...,50.0,Christel Koranda,North Central,6,4S,64W,Active,20.00,"TRACT 16, COLOMEADOW ACRES",16


In [117]:
merged_df['Township/Range'] = merged_df['Township'].astype(str) + ' ' + merged_df['Range'].astype(str)
merged_df = merged_df.drop(columns=['Township', 'Range'])

merged_df.head(10)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Legal Description_x,Lease Terms (Years),Administrator,District,Meridian,Lease Status,Total Lease Acreage,Lease Legal Description(s),Section(s),Township/Range
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,Active,960.00,"E2, ALL, E2","10, 36, 10",5S 59W
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,ALL,15.0,Rachel Turner,North Central,6,Active,960.00,"E2, ALL, E2","10, 36, 10",5S 59W
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,Active,960.00,"E2, ALL, E2","10, 36, 10",5S 59W
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,Active,47859.79,See Lease Documentation,See Lease Documentation,15S 62W
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,Active,47859.79,See Lease Documentation,See Lease Documentation,16S 62W
5,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004769,6-17S-62W-1-Surface-El Paso-School,...,"LTS 2-4, SWNE, S2NW, SW, SE",25.0,Justin Osborne,South Central,6,Active,47859.79,See Lease Documentation,See Lease Documentation,17S 62W
6,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM000142,6-15S-62W-2-Surface-El Paso-School,...,"LTS 1-2, S2NE, SE",25.0,Justin Osborne,South Central,6,Active,47859.79,See Lease Documentation,See Lease Documentation,15S 62W
7,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004636,6-16S-62W-2-Surface-El Paso-School,...,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,Active,47859.79,See Lease Documentation,See Lease Documentation,16S 62W
8,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004770,6-17S-62W-2-Surface-El Paso-School,...,ALL,25.0,Justin Osborne,South Central,6,Active,47859.79,See Lease Documentation,See Lease Documentation,17S 62W
9,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004637,6-16S-62W-3-Surface-El Paso-School,...,ALL,25.0,Justin Osborne,South Central,6,Active,47859.79,See Lease Documentation,See Lease Documentation,16S 62W
